In [ ]:
import os
import pandas as pd
import numpy as np
from neo4j import Query, GraphDatabase, RoutingControl, Result
from graphdatascience import GraphDataScience 

## Database connection

In [ ]:
DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "neo4j" # Have to be neo4j for neo4j aura (but keep it, good for testing on local dev env)
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [28]:
for i in [1,2]:
    with driver.session(database=DB_NAME).begin_transaction() as tx:
        try:
            result = tx.run('''
                            create(n:Person{id:$id})
                            return count(*) as nodes_created
                            ''', id=i)
            print(result.data())
        except Exception as e: 
            print(e)
        finally:
            if not tx.closed():
                tx.rollback()


{code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(0) already exists with label `Person` and property `id` = 1}
[{'nodes_created': 1}]


In [33]:
records, summary, keys = driver.execute_query(
    ''' 
        unwind $rows as row
        merge(n:Person{id:row.id})
        set n.name = row.name
        return count(*) as nodes_created
    ''',
    database_=DB_NAME,
    routing_=RoutingControl.WRITE,
    rows = [{'id':1, 'name':'A'} , {'id':2, 'name': 'B'} ]
)
print(summary.counters)

{'_contains_updates': True, 'properties_set': 2}
